In [10]:
from openai import OpenAI
import json
client = OpenAI(api_key="sample")

In [11]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

In [12]:
from datetime import datetime,timedelta
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "IND",
        "flight": "XY54J",
    }

    return json.dumps(flight_info)

In [13]:
user_prompt = "When's the next flight from new nashik to delhi?"

In [14]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": user_prompt
    }
      ],
    functions=function_descriptions,
    function_call="auto", 
    
)

In [15]:
params=json.loads(response2.choices[0].message.function_call.arguments)
chosen_function=eval(response2.choices[0].message.function_call.name)
flight = chosen_function(**params)


In [16]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user","content": user_prompt},
      {"role": "function", "name": response2.choices[0].message.function_call.name, "content": flight}
    ],
)

In [17]:
response3.choices[0].message.content

'The next flight from New Nashik (ISK) to Delhi (DEL) is scheduled for August 6, 2024. The flight is operated by IND Airlines with flight number XY54J.'